Stage 2 training on 224*224

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd drive/MyDrive/Project/COVID-19-master

/content/drive/.shortcut-targets-by-id/148tduUm3_N2YmcWCKycU5qdfNmgLJccA/Project/COVID-19-master


In [4]:
# #importing 
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [5]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization

In [6]:
from os import listdir
from os.path import isfile, join
xtrain_224=np.load('data/x_train.npy')
ytrain = np.load('data/y_train.npy')
print("RUN HOGAYA")
    
xtest = np.load('data/x_test.npy')
ytest = np.load('data/y_test.npy')

RUN HOGAYA


224 wala

In [7]:
def confusion_matrix_info(y_true, y_pred, labels=['normal', 'pneumonia',  'COVID-19'],
                          title='confusion matrix'):
    import seaborn as sns
    import pandas as pd
    from sklearn.metrics import confusion_matrix, f1_score
    import matplotlib.pyplot as plt
    C2 = confusion_matrix(y_true, y_pred)
    C = pd.DataFrame(C2, columns=labels, index=labels)
    m, _ = C2.shape
    for i in range(m):
        precision = C2[i, i] / sum(C2[:, i])
        recall = C2[i, i] / sum(C2[i, :])
        f1 = 2 * precision * recall / (precision + recall)
        print('In class {}:\t total samples: {}\t true predict samples: {}\t'
              'acc={:.4f},\trecall={:.4f},\tf1-score={:.4f}'.format(
            labels[i], sum(C2[i, :]), C2[i, i], precision, recall, f1))
    print('-' * 100, '\n', 'average f1={:.4f}'.format(f1_score(y_true, y_pred, average='micro')))
 
    f, ax = plt.subplots()
    sns.heatmap(C, annot=True, ax=ax, cmap=plt.cm.binary)
    ax.set_title(title)
    ax.set_xlabel('predict')
    ax.set_ylabel('true')
    plt.savefig(title+'.jpg')

In [8]:

resnet_weights_path = 'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
NUM_CLASSES = 3
DENSE_LAYER_ACTIVATION = 'softmax'



In [9]:
#resnet_weights_path = 'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
NUM_CLASSES = 3
DENSE_LAYER_ACTIVATION = 'softmax'

def Second_keras_model():
    model2 = Sequential()

    # 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
    # NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
    model2.add(ResNet50(include_top = False,input_shape=(224,224,3), pooling = 'avg',weights = resnet_weights_path))
    model2.add(BatchNormalization())

    # 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
    model2.add(Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION))
    model2.load_weights("model1.h5")

    # Say not to train first layer (ResNet) model as it is already trained
    
    return model2

In [13]:
def train(xtrain224, y):
    y = tf.keras.utils.to_categorical(y, 3)
    #y_test = tf.keras.utils.to_categorical(y_test, 3)
    #TRAINING ON 224*224 WITH FREEZED RESNET LAYER
    model2 = Second_keras_model()
    #print(model2.summary)
    model2.layers[0].trainable = False
    
    print(model2.summary())
    opt=tf.keras.optimizers.Adam(lr=0.0001)
    model2.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    hist = model2.fit(xtrain224, y, batch_size=32,  epochs=3, verbose=1)


    #FINE TUNING ON 224 WITH UNFREEZED RESNET LAYER
    model2=model2
    model2.layers[0].trainable = True
    print(model2.summary())
    opt=tf.keras.optimizers.Adam(lr=0.0001)
    model2.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    hist = model2.fit(xtrain224, y, batch_size=32,  epochs=5, verbose=1)
    model2.save('model2.h5')

    y_pred = model3.predict(xtrain224)
    confusion_matrix_info(np.argmax(y, axis=1), np.argmax(y_pred, axis=1),title='confusion_matrix_train')

In [11]:
def test(xt, yt):
    model = tf.keras.models.load_model('model2.h5')
    
    y_pred = model.predict(xt)
    confusion_matrix_info(yt, np.argmax(y_pred, axis=1),title='confusion_matrix_test')

In [ ]:
train(xtrain_224, ytrain)

In [16]:
test(xtest, ytest)

OSError: ignored